In [46]:
#import libraries
import pandas as pd
import string
from nltk.corpus import stopwords 

In [47]:
#get spam data collection using pandas
spam_data = pd.read_csv('SpamCollection',sep = '\t',names = ["response","message"])

In [48]:
#view first 5 columns
spam_data.head()

,response,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [49]:
#use describe method to get more info
spam_data.describe()

,response,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [50]:
#view response using group by and describe method
spam_data.groupby('response').describe()

message                                                            \
           count unique                                                top   
response                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
response       
ham        30  
spam        4

In [51]:
#verfy length and add it as a new column(feature)
spam_data['length'] = spam_data['message'].apply(len)

In [52]:
#view new column
spam_data.head()

,response,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [53]:
#function to remove stopwords
def remove_stopwords(msg):
    #check character to see if there are panctuations
    no_punctuation = [char for char in msg if char not in string.punctuation]
    #now form the sentence
    no_punctuation  = ''.join(no_punctuation)
    #now eliminate any stopwords
    return [word for word in no_punctuation.split(' ',1) if word.lower() not in stopwords.words('english')]
    

In [54]:
#verify if function is working
spam_data['message'].head(5).apply(remove_stopwords)

0    [Go, until jurong point crazy Available only i...
1                           [Ok, lar Joking wif u oni]
2    [Free, entry in 2 a wkly comp to win FA Cup fi...
3       [U, dun say so early hor U c already then say]
4    [Nah, I dont think he goes to usf he lives aro...
Name: message, dtype: object

In [55]:
#start text processing with vectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [61]:
#bag of words by applying the function to fit data(message) into it
bag_of_words = CountVectorizer(analyzer = remove_stopwords).fit(spam_data['message'])

In [62]:
#print bag of words length in vocabulary_ attribute
print (len(bag_of_words.vocabulary_))

6239


In [60]:
#store bag of words for messages using transform method
message_bagofwords = bag_of_words.transform(spam_data['message'])

In [63]:
#apply tfidf transformer and fit the message_bagofwords into it
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(message_bagofwords)

In [64]:
#print shape of tfidf
message_tfidf = tfidf_transformer.transform(message_bagofwords)

In [65]:
#print shape
message_tfidf.shape

(5572, 6239)

In [92]:
#Use naive base model to detect spam and fit the tf idf data to it
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(message_tfidf,spam_data['response'])

In [93]:
# check model for the predicted and expected value
# passing second message for testing
message = spam_data['message'][4]
#transform the message usig bag_of_words
bag_of_words_for_msg = bag_of_words.transform([message])
#perform tfidf transformation
tfidfmsg = tfidf_transformer.transform(bag_of_words_for_msg)

print ('predicted',spam_detect_model.predict(tfidfmsg)[0])
print ('expected',spam_data.response[4])

predicted ham
expected ham
